# Set constant

In [1]:
max_len = 20 # max sentence size

# Load libaries

In [2]:
from freq_utils import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

from tensorflow.keras.utils import to_categorical

pd.options.display.max_colwidth = 200

# Load dataset

In [3]:
df0 = pd.read_csv('data/TrueOrganized.csv')
df1 = pd.read_csv('data/FakeOrganized.csv')
df0['label'] = 0
df1['label'] = 1

In [4]:
df = df0.sample(2)
display(df)

,Unnamed: 0,title,pos,cleaned_words,cleaned_pos,minimal_words,org_title,label
11731,11731,Philippine leader declares unilateral ceasefire for Christmas,"[('Philippine', 'JJ'), ('leader', 'NN'), ('declares', 'VBZ'), ('unilateral', 'JJ'), ('ceasefire', 'NN'), ('for', 'IN'), ('Christmas', 'NNP')]",philippine leader declares unilateral ceasefire for christmas,JJ NN VBZ JJ NN IN NNP,philippine leader declare unilateral ceasefire christmas,Philippine leader declares unilateral ceasefire for Christmas,0
12709,12709,Hamas calls for Palestinian uprising over Trump's Jerusalem plan,"[('Hamas', 'NNP'), ('calls', 'VBZ'), ('for', 'IN'), ('Palestinian', 'JJ'), ('uprising', 'NN'), ('over', 'IN'), ('Trump', 'NNP'), (""'s"", 'POS'), ('Jerusalem', 'NNP'), ('plan', 'NN')]",hamas calls for palestinian uprising over trump s jerusalem plan,NNP VBZ IN JJ NN IN NNP POS NNP NN,hamas call palestinian uprising trump jerusalem plan,Hamas calls for Palestinian uprising over Trump's Jerusalem plan,0


# Make dictionaries

### Pretrained word embeddings
- Word to index
- Word to vector

In [5]:
word_to_index, index_to_word, word_to_vector = get_pretrained_embedding()

### PoS tag encodings
- PoS word to index

In [6]:
df = pd.concat([df0.cleaned_pos, df1.cleaned_pos])

pos_set = set()
for x in list(df.str.lower().str.split()):
    pos_set.update(x)

pos_list = list(pos_set)
pos_to_index = { pos_list[i]: i for i in range(len(pos_list)) }

print(pos_to_index)

{'prp$': 0, 'rbr': 1, 'ex': 2, 'md': 3, 'dt': 4, '.': 5, 'in': 6, 'jj': 7, 'fw': 8, '$': 9, 'nns': 10, 'uh': 11, 'vbd': 12, 'wp': 13, "''": 14, 'wrb': 15, 'vb': 16, 'cc': 17, 'rbs': 18, 'nn': 19, 'to': 20, 'jjr': 21, 'cd': 22, 'prp': 23, 'rp': 24, 'wdt': 25, 'pos': 26, 'pdt': 27, 'vbz': 28, 'vbp': 29, ':': 30, 'nnp': 31, 'rb': 32, 'wp$': 33, 'sym': 34, 'vbn': 35, 'nnps': 36, 'vbg': 37, 'jjs': 38}


In [7]:
len(pos_to_index)

39

# Train/dev/test split

In [8]:
train, dev, test = train_dev_test_split([df0, df1], m=1000, class_column='label', 
                                    class_balance=True, r_dev=0.2, r_test=0.2, rand_state=42)

# Define models

In [9]:
def simple_NB(train,dev,test,Xname='title',Yname='label'):
    
    train = pd.concat([train,dev])
    
    X_train = train[Xname].tolist()
    Y_train = train[Yname].tolist()

    X_test = test[Xname].tolist()
    Y_test = test[Yname].tolist()
    
    counter = CountVectorizer()

    counter.fit(X_train+X_test)

    train_counts = counter.transform(X_train)
    test_counts = counter.transform(X_test)

    #print(counter.vocabulary_)

    classifier = MultinomialNB()
    classifier.fit(train_counts,Y_train)
    
    predict = classifier.predict(test_counts)
    
    proba = classifier.predict_proba(test_counts)
    
    model_name = 'Naive Bayse - '+Xname
    
    
    return model_name, predict, Y_test, proba, classifier, counter

In [10]:
def model_FNN(input_shape, word_to_index, word_to_vector, n_class=2, trainable=False):
    '''
    input_shape: (max_len,)
    word_to_index: word to index dictionary
    word_to_vector: word to embedding vector dictionary

    return model

    then
    X: Indices of a sentence (m, max_len)
    Y: Class probability, one hot vector (m, # classes)
    '''

    # Input layer
    # X_indices (m, max_len)
    X_input = tfl.Input(shape=input_shape, dtype='int32')
    
    # Embedding layer
    embedding_layer = pretrained_embedding_layer(word_to_vector, word_to_index, trainable=trainable)
    X = embedding_layer(X_input)   

    # Take average
    # Get embedding vector dimension
    emb_dim = X.shape[2]
    # Make a list from slice
    X_avg = [ X[:,:,i] for i in range(emb_dim) ]
    # Take average of embedding vector
    X = tf.keras.layers.Average()(X_avg)


    # Masking layer
    # skip zero vector words
    X = tfl.Masking(mask_value=0.)(X)

    # Linear+ReLu layer
    X = tfl.Dense(units = 128, activation='relu')(X)
    X = tfl.Dropout(rate = 0.4)(X)  

    # Linear+ReLu layer
    X = tfl.Dense(units = 64, activation='relu')(X)
    X = tfl.Dropout(rate = 0.4)(X) 

    # Linear+ReLu layer
    X = tfl.Dense(units = 32, activation='relu')(X)
    X = tfl.Dropout(rate = 0.2)(X) 

    # Linear+Softmax layer
    # Output: y (m, # classes), probability of each class
    X = tfl.Dense(units = n_class, activation='softmax')(X)

    # Model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

In [11]:
def model_LSTM(input_shape, word_to_index, word_to_vector=False, n_class=2, trainable=False):
    '''
    input_shape: (max_len,) or (max_len, num_cat)
    word_to_index: word to index dictionary, False for one hot encoding
    word_to_vector: word to embedding vector dictionary, False for one hot encoding

    return model

    then
    X: Indices of a sentence (m, max_len)
    Y: Class probability, one hot vector (m, # classes)
    '''

    # Input layer
    # X_oh (m, max_len, num_cat)
    # X_indices (m, max_len)
    X_input = tfl.Input(shape=input_shape, dtype='int32')

    # Embedding layer
    #embedding_layer = pretrained_embedding_layer(word_to_vector, word_to_index, trainable=trainable)
    #X = embedding_layer(X_indices)   

    # By default, assumes one hot vector input
    # If word_to_index, word_to_vector is provided, add an embedding layer
    X = X_input
    if bool(word_to_vector):
        # Word embeding
        # Output: (m, max_len, emb_dim)
        # Embedding layer
        embedding_layer = pretrained_embedding_layer(word_to_vector, word_to_index, trainable=trainable)
        X = embedding_layer(X)      
        # Masking layer
        # skip zero vector words
        X = tfl.Masking(mask_value=0.)(X)
        
    else:
        X = tf.dtypes.cast(X, tf.float32)

    # LSTM layer
    # Output: a[1] (m, max_len, 128 hidden unit), batch of sequences
    X = tfl.LSTM(units = 128, return_sequences= True)(X)
    X = tfl.Dropout(rate = 0.5 )(X) 

    # LSTM layer
    # Output: a[2]<max_len> (m, 128 hidden unit)
    X = tfl.LSTM(units = 128)(X)
    X = tfl.Dropout(rate = 0.5)(X)  

    # Linear layer
    # Output: a[3] (m, # classes)
    X = tfl.Dense(units = n_class)(X)

    # Softmax layer
    # Output: y (m, # classes), probability of each class
    X = tfl.Activation('softmax')(X)

    # Model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

# Model wrapper functions

In [15]:
def run_multiple_NB(model_vars,train,dev,test,Yname='label'):

    results = []

    for var in model_vars:
        
        Xname = var
        
        model_name, y_pred, y_true, proba, classifier, counter = \
            simple_NB(train,dev,test,Xname=Xname,Yname=Yname)
        
        x = test[Xname].to_numpy()
        
        results.append([model_name, x, y_pred, y_true, proba, classifier, counter])
        
    return results

In [16]:
def run_multiple_NN(model_vars, train, dev, test, pos_to_index, word_to_index=False, word_to_vector=False, 
                        Yname='label', max_len=20, n_class=2, 
                        epochs = 20, batch_size = 32, patience=2, trainable=False):
    
    
    results = []

    for i in range(len(model_vars)):
            
        model_name = model_vars[i][0]
        func_model = model_vars[i][1]
        Xname = model_vars[i][2]
        use_embeddings = model_vars[i][3]
        
        w2i = False
        w2v = False
        X_shape = False
        
        if use_embeddings:
            w2i = word_to_index        
            w2v = word_to_vector
            X_shape = (max_len, )
        else:
            w2i = pos_to_index
            X_shape = (max_len, len(pos_to_index))

        _, _, X_train_indices, _, Y_train_oh = dataframe_to_arrays(train, w2i, max_len, Xname=Xname)
        _, _, X_dev_indices,   _, Y_dev_oh   = dataframe_to_arrays(dev, w2i, max_len, Xname=Xname)
        index, _, X_test_indices, Y_test, _  = dataframe_to_arrays(test, w2i, max_len, Xname=Xname)

        
        X = False
        if use_embeddings:
            X = [X_train_indices, X_dev_indices, X_test_indices]
        else:
            X = [to_categorical(X_train_indices, num_classes=len(pos_to_index)), 
                 to_categorical(X_dev_indices, num_classes=len(pos_to_index)), 
                 to_categorical(X_test_indices, num_classes=len(pos_to_index))]            

        
        Y = [Y_train_oh, Y_dev_oh, Y_test]

        
    
        print('should match all: ', X_shape, X[0].shape, X[1].shape, X[2].shape)
    
        model = func_model(X_shape, w2i, w2v, n_class, trainable)
        model.summary()
                    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)  
        
        history = False
        
        if patience :
            history = model.fit(X[0], Y[0], 
                                epochs = epochs, batch_size = batch_size, shuffle=True, 
                                validation_data=(X[1], Y[1]),
                                callbacks=[es])
        else:
            history = model.fit(X[0], Y[0], 
                                epochs = epochs, batch_size = batch_size, shuffle=True, 
                                validation_data=(X[1], Y[1]))
            
        proba = model.predict(X[2])
        y_pred = [np.argmax(proba[i]) for i in range(len(proba))]
        y_true = Y[2]
        x = X[2]
        
        results.append([model_name, x, y_pred, y_true, proba, model, history])
    
    return results

# Run models

In [17]:
# Naive Bayes
#simple_NB(train,dev,test,Xname='title',Yname='label') 
nn_vars = ['cleaned_words','minimal_words','cleaned_pos' ]
res_nb =  run_multiple_NB(nn_vars,train,dev,test,Yname='label')
    

# NN
nn_vars = [#['LSTM clean words', model_LSTM, 'cleaned_words', True], 
              ['LSTM clean pos', model_LSTM, 'cleaned_pos', False],
            ['FNN clean words', model_FNN, 'cleaned_words', True]]

res_nn = run_multiple_NN(nn_vars, train, dev, test,
                         pos_to_index, word_to_index, word_to_vector, 
                         Yname='label', max_len=max_len, n_class=2,
                         epochs = 2, batch_size = 32, patience=2, trainable=False)

# Add NB and NN
results = res_nb + res_nn


should match all:  (20, 39) (600, 20, 39) (200, 20, 39) (200, 20, 39)


2022-02-19 00:53:45.835874: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 39)]          0         
                                                                 
 tf.cast (TFOpLambda)        (None, 20, 39)            0         
                                                                 
 lstm (LSTM)                 (None, 20, 128)           86016     
                                                                 
 dropout (Dropout)           (None, 20, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258   

19/19 [==============================] - 4s 32ms/step - loss: 0.6950 - accuracy: 0.4867 - val_loss: 0.6929 - val_accuracy: 0.5400
Epoch 2/2
19/19 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5117 - val_loss: 0.6929 - val_accuracy: 0.5200


# Print result

In [18]:
def print_result(results):
    
    for result in results:
        
        model_name, x, y_pred, y_true, proba = result[:5]
        
        print(model_name)
        print('accuracy: ',accuracy_score(y_true, y_pred))
        print('precision: ',precision_score(y_true, y_pred))
        print('recall: ',recall_score(y_true, y_pred))
        print('f1: ',f1_score(y_true, y_pred))
        print('\n')

In [19]:
print_result(results)

#print(len(res_nn))

Naive Bayse - cleaned_words
accuracy:  0.92
precision:  0.9381443298969072
recall:  0.900990099009901
f1:  0.9191919191919191


Naive Bayse - minimal_words
accuracy:  0.91
precision:  0.9368421052631579
recall:  0.8811881188118812
f1:  0.9081632653061223


Naive Bayse - cleaned_pos
accuracy:  0.93
precision:  0.9393939393939394
recall:  0.9207920792079208
f1:  0.9300000000000002


LSTM clean pos
accuracy:  0.935
precision:  0.9489795918367347
recall:  0.9207920792079208
f1:  0.934673366834171


FNN clean words
accuracy:  0.45
precision:  0.4117647058823529
recall:  0.2079207920792079
f1:  0.27631578947368424


